# کلاؤڈ میں ڈیٹا سائنس: "Azure ML SDK" کا طریقہ

## تعارف

اس نوٹ بک میں، ہم سیکھیں گے کہ Azure ML SDK کا استعمال کرتے ہوئے ماڈل کو ٹرین، ڈپلائے اور کنزیوم کیسے کیا جائے۔

پیشگی ضروریات:
1. آپ نے ایک Azure ML ورک اسپیس بنایا ہے۔
2. آپ نے [ہارٹ فیلئر ڈیٹاسیٹ](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) کو Azure ML میں لوڈ کیا ہے۔
3. آپ نے اس نوٹ بک کو Azure ML اسٹوڈیو میں اپلوڈ کیا ہے۔

اگلے مراحل یہ ہیں:

1. موجودہ ورک اسپیس میں ایک تجربہ (Experiment) بنائیں۔
2. ایک کمپیوٹ کلسٹر بنائیں۔
3. ڈیٹاسیٹ لوڈ کریں۔
4. AutoMLConfig کا استعمال کرتے ہوئے آٹو ایم ایل کو کنفیگر کریں۔
5. آٹو ایم ایل تجربہ چلائیں۔
6. نتائج کو دریافت کریں اور بہترین ماڈل حاصل کریں۔
7. بہترین ماڈل کو رجسٹر کریں۔
8. بہترین ماڈل کو ڈپلائے کریں۔
9. اینڈ پوائنٹ کو کنزیوم کریں۔

## Azure Machine Learning SDK کے مخصوص امپورٹس


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## ورک اسپیس کو شروع کریں  
محفوظ شدہ کنفیگریشن سے ورک اسپیس آبجیکٹ کو شروع کریں۔ یقینی بنائیں کہ کنفیگریشن فائل .\config.json پر موجود ہے۔


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## ایک Azure ML تجربہ بنائیں

آئیے اس ورک اسپیس میں ایک تجربہ بناتے ہیں جسے ہم نے ابھی ترتیب دیا ہے، جس کا نام 'aml-experiment' ہے۔


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## کمپیوٹ کلسٹر بنائیں  
آپ کو اپنے AutoML رن کے لیے ایک [کمپیوٹ ٹارگٹ](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) بنانا ہوگا۔  


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## ڈیٹا  
یقینی بنائیں کہ آپ نے ڈیٹا سیٹ Azure ML پر اپلوڈ کر دیا ہے اور کہ کلید کا نام ڈیٹا سیٹ کے نام کے ساتھ ایک جیسا ہے۔


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## آٹو ایم ایل تشکیل


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## آٹو ایم ایل رن


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## بہترین ماڈل کو تعینات کریں

بہترین ماڈل کو تعینات کرنے کے لیے درج ذیل کوڈ چلائیں۔ آپ Azure ML پورٹل میں تعیناتی کی حالت دیکھ سکتے ہیں۔ یہ مرحلہ چند منٹ لے سکتا ہے۔


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## اینڈ پوائنٹ کا استعمال کریں  
آپ درج ذیل ان پٹ نمونے میں ان پٹ شامل کر سکتے ہیں۔  


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
